## **Complete Code For Getting The Weather Data**

In [0]:

import requests
import json

# Function to handle the API response
def handle_response(response):
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error: {response.status_code}, {response.text}"   

# Function to get current weather and air quality data
def get_current_weather(base_url, api_key, location):
    current_weather_url = f"{base_url}/current.json"
    params = {
        'key': api_key,
        'q': location,
        "aqi": 'yes'
    }
    response = requests.get(current_weather_url, params=params)
    return handle_response(response)   

# Function to get Forecast Data
def get_forecast_weather(base_url, api_key, location, days):
    forecast_url = f"{base_url}/forecast.json"
    params = {
        "key": api_key,
        "q": location,
        "days": days,
    }
    response = requests.get(forecast_url, params=params)
    return handle_response(response)

# Function to get Alerts
def get_alerts(base_url, api_key, location):
    alerts_url = f"{base_url}/alerts.json"
    params = {
        'key': api_key,
        'q': location,
        "alerts": 'yes'
    }
    response = requests.get(alerts_url, params=params)
    return handle_response(response)

# Flatten and merge the data
def flatten_data(current_weather, forecast_weather, alerts):
    location_data = current_weather.get("location", {})
    current = current_weather.get("current", {})
    condition = current.get("condition", {})
    air_quality = current.get("air_quality", {})
    forecast = forecast_weather.get("forecast", {}).get("forecastday", [])
    alert_list = alerts.get("alerts", {}).get("alert", [])

    flattened_data = {
        'name': location_data.get('name'),
        'region': location_data.get('region'),
        'country': location_data.get('country'),
        'tz_id': location_data.get('tz_id'),
        'lat': location_data.get('lat'),
        'lon': location_data.get('lon'),
        'localtime': location_data.get('localtime'),
        'temp_c': current.get('temp_c'),
        'is_day': current.get('is_day'),
        'condition_text': condition.get('text'),
        'condition_icon': condition.get('icon'),
        'wind_kph': current.get('wind_kph'),
        'wind_degree': current.get('wind_degree'),
        'wind_dir': current.get('wind_dir'),
        'pressure_in': current.get('pressure_in'),
        'precip_in': current.get('precip_in'),
        'humidity': current.get('humidity'),
        'cloud': current.get('cloud'),
        'feelslike_c': current.get('feelslike_c'),
        'uv': current.get('uv'),
        'air_quality': {
            'co': air_quality.get('co'),
            'no2': air_quality.get('no2'),
            'o3': air_quality.get('o3'),
            'so2': air_quality.get('so2'),
            'pm2_5': air_quality.get('pm2_5'),
            'pm10': air_quality.get('pm10'),
            'us-epa-index': air_quality.get('us-epa-index'),
            'gb-defra-index': air_quality.get('gb-defra-index')
        },
        'alerts': [
            {
                'headline': alert.get('headline'),
                'msgtype': alert.get('msgtype'),
                'severity': alert.get('severity'),
                'urgency': alert.get('urgency'),
                'areas': alert.get('areas'),
                'event': alert.get('event' ),
                'note': alert.get('note'),
                'description': alert.get('desc'),
                'instruction': alert.get('instruction')
            }
            for alert in alert_list
        ],
        'forecast': [
            {
                'date': day.get('date'),
                'maxtemp_c': day.get('day', {}).get('maxtemp_c'),
                'mintemp_c': day.get('day', {}).get('mintemp_c'),
                'condition': day.get('day', {}).get('condition', {}).get('text')
            }
            for day in forecast
        ]
    }
    return flattened_data

# Main program
def fetch_weather_data():

    base_url = "http://api.weatherapi.com/v1/"
    location = "Nairobi"
    weatherapikey = dbutils.secrets.get(scope="key-vault-scope", key="weather-api-key")

    # Get data from API
    current_weather = get_current_weather(base_url, weatherapikey, location)
    forecast_weather = get_forecast_weather(base_url, weatherapikey, location, 3)
    alerts = get_alerts(base_url, weatherapikey, location)

    # Flatten and merge data
    merged_data = flatten_data(current_weather, forecast_weather, alerts)
    print("Weather Data:", json.dumps(merged_data, indent=3))

# Calling the Main Program
fetch_weather_data()

Weather Data: {
   "name": "Nairobi",
   "region": "Nairobi Area",
   "country": "Kenya",
   "tz_id": "Africa/Nairobi",
   "lat": -1.2833,
   "lon": 36.8167,
   "localtime": "2025-07-15 22:09",
   "temp_c": 18.0,
   "is_day": 0,
   "condition_text": "Partly cloudy",
   "condition_icon": "//cdn.weatherapi.com/weather/64x64/night/116.png",
   "wind_kph": 5.4,
   "wind_degree": 75,
   "wind_dir": "ENE",
   "pressure_in": 30.27,
   "precip_in": 0.0,
   "humidity": 68,
   "cloud": 75,
   "feelslike_c": 18.0,
   "uv": 0.0,
   "air_quality": {
      "co": 695.6,
      "no2": 26.27,
      "o3": 85.0,
      "so2": 11.285,
      "pm2_5": 48.47,
      "pm10": 56.795,
      "us-epa-index": 3,
      "gb-defra-index": 6
   },
   "alerts": [],
   "forecast": [
      {
         "date": "2025-07-15",
         "maxtemp_c": 22.7,
         "mintemp_c": 12.4,
         "condition": "Patchy rain nearby"
      },
      {
         "date": "2025-07-16",
         "maxtemp_c": 21.5,
         "mintemp_c": 13.3,
  

## **Sending Weather Data To Event Hub After Every 30sec**

In [0]:
from azure.eventhub import EventHubProducerClient, EventData
import json
import requests
from datetime import datetime, timedelta

# Event Hub configuration
EVENT_HUB_NAME = "weather-eventhub"
eventhub_connection_string = dbutils.secrets.get(scope="key-vault-scope", key="event-hub-connection-string")
weatherapikey = dbutils.secrets.get(scope="key-vault-scope", key="weather-api-key")

# Initialize the Event Hub producer
producer = EventHubProducerClient.from_connection_string(conn_str=eventhub_connection_string, eventhub_name=EVENT_HUB_NAME)

# Function to send events to Event Hub
def send_event(event):
    event_data_batch = producer.create_batch()
    event_data_batch.add(EventData(json.dumps(event)))
    producer.send_batch(event_data_batch)

# Function to handle the API response
def handle_response(response):
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error: {response.status_code}, {response.text}"   

# Function to get current weather and air quality data
def get_current_weather(base_url, api_key, location):
    current_weather_url = f"{base_url}/current.json"
    params = {
        'key': api_key,
        'q': location,
        "aqi": 'yes'
    }
    response = requests.get(current_weather_url, params=params)
    return handle_response(response)   

# Function to get Forecast Data
def get_forecast_weather(base_url, api_key, location, days):
    forecast_url = f"{base_url}/forecast.json"
    params = {
        "key": api_key,
        "q": location,
        "days": days,
    }
    response = requests.get(forecast_url, params=params)
    return handle_response(response)

# Function to get Alerts
def get_alerts(base_url, api_key, location):
    alerts_url = f"{base_url}/alerts.json"
    params = {
        'key': api_key,
        'q': location,
        "alerts": 'yes'
    }
    response = requests.get(alerts_url, params=params)
    return handle_response(response)

# Flatten and merge the data
def flatten_data(current_weather, forecast_weather, alerts):
    location_data = current_weather.get("location", {})
    current = current_weather.get("current", {})
    condition = current.get("condition", {})
    air_quality = current.get("air_quality", {})
    forecast = forecast_weather.get("forecast", {}).get("forecastday", [])
    alert_list = alerts.get("alerts", {}).get("alert", [])

    flattened_data = {
        'name': location_data.get('name'),
        'region': location_data.get('region'),
        'country': location_data.get('country'),
        'tz_id': location_data.get('tz_id'),
        'lat': location_data.get('lat'),
        'lon': location_data.get('lon'),
        'localtime': location_data.get('localtime'),
        'temp_c': current.get('temp_c'),
        'is_day': current.get('is_day'),
        'condition_text': condition.get('text'),
        'condition_icon': condition.get('icon'),
        'wind_kph': current.get('wind_kph'),
        'wind_degree': current.get('wind_degree'),
        'wind_dir': current.get('wind_dir'),
        'pressure_in': current.get('pressure_in'),
        'precip_in': current.get('precip_in'),
        'humidity': current.get('humidity'),
        'cloud': current.get('cloud'),
        'feelslike_c': current.get('feelslike_c'),
        'uv': current.get('uv'),
        'air_quality': {
            'co': air_quality.get('co'),
            'no2': air_quality.get('no2'),
            'o3': air_quality.get('o3'),
            'so2': air_quality.get('so2'),
            'pm2_5': air_quality.get('pm2_5'),
            'pm10': air_quality.get('pm10'),
            'us-epa-index': air_quality.get('us-epa-index'),
            'gb-defra-index': air_quality.get('gb-defra-index')
        },
        'alerts': [
            {
                'headline': alert.get('headline'),
                'msgtype': alert.get('msgtype'),
                'severity': alert.get('severity'),
                'urgency': alert.get('urgency'),
                'areas': alert.get('areas'),
                'event': alert.get('event' ),
                'note': alert.get('note'),
                'description': alert.get('desc'),
                'instruction': alert.get('instruction')
            }
            for alert in alert_list
        ],
        'forecast': [
            {
                'date': day.get('date'),
                'maxtemp_c': day.get('day', {}).get('maxtemp_c'),
                'mintemp_c': day.get('day', {}).get('mintemp_c'),
                'condition': day.get('day', {}).get('condition', {}).get('text')
            }
            for day in forecast
        ]
    }
    return flattened_data


def fetch_weather_data():

    base_url = "http://api.weatherapi.com/v1/"
    location = "Nairobi"
    weatherapikey = dbutils.secrets.get(scope="key-vault-scope", key="weather-api-key")

    # Get data from API
    current_weather = get_current_weather(base_url, weatherapikey, location)
    forecast_weather = get_forecast_weather(base_url, weatherapikey, location, 3)
    alerts = get_alerts(base_url, weatherapikey, location)

    # Flatten and merge data
    merged_data = flatten_data(current_weather, forecast_weather, alerts)
    return merged_data

# Function to process each batch of streaming data
last_sent_time = datetime.now() - timedelta(seconds=30)  # Initialize last sent time


# Main program
def process_batch(batch_df, batch_id):
    global last_sent_time
    try:
        # Get current time
        current_time = datetime.now()
        
        # Check if 30 seconds have passed since last event was sent
        if (current_time - last_sent_time).total_seconds() >= 4:
            # Fetch weather data
            weather_data = fetch_weather_data()
            
            # Send the weather data (current weather part)
            send_event(weather_data)

            # Update last sent time
            last_sent_time = current_time
            print(f'Event Sent at {last_sent_time}')

    except Exception as e:
        print(f"Error sending events in batch {batch_id}: {str(e)}")
        raise e

# Set up a streaming source (for example, rate source for testing purposes)
streaming_df = spark.readStream.format("rate").option("rowsPerSecond", 1).load()

# Write the streaming data using foreachBatch to send weather data to Event Hub
query = streaming_df.writeStream.foreachBatch(process_batch).start()

query.awaitTermination()

# Close the producer after termination
producer.close()


Event Sent at 2025-07-16 20:50:10.222217
Event Sent at 2025-07-16 20:50:14.264885
Event Sent at 2025-07-16 20:50:19.243627
Event Sent at 2025-07-16 20:50:23.360437
Event Sent at 2025-07-16 20:50:28.305080
Event Sent at 2025-07-16 20:50:33.174840
Event Sent at 2025-07-16 20:50:37.323973
Event Sent at 2025-07-16 20:50:42.174969
Event Sent at 2025-07-16 20:50:46.329117
Event Sent at 2025-07-16 20:50:51.266143
Event Sent at 2025-07-16 20:50:56.219499
Event Sent at 2025-07-16 20:51:01.216280
Event Sent at 2025-07-16 20:51:05.333870
Event Sent at 2025-07-16 20:51:10.149394
Event Sent at 2025-07-16 20:51:14.295290
Event Sent at 2025-07-16 20:51:19.122592
Event Sent at 2025-07-16 20:51:23.208714
Event Sent at 2025-07-16 20:51:27.380557
Event Sent at 2025-07-16 20:51:32.270153
Event Sent at 2025-07-16 20:51:37.113853
Event Sent at 2025-07-16 20:51:41.130852
Event Sent at 2025-07-16 20:51:45.432195
Event Sent at 2025-07-16 20:51:50.302471
Event Sent at 2025-07-16 20:51:55.177159
Event Sent at 20

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:132)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:132)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can